In [7]:
#!ls -l Jingjing/pca/*.parquet
!ls -1d Jingjing/pca/* | xargs -I{} du {} -sh

344G	Jingjing/pca/100cases.parquet
45G	Jingjing/pca/lpd_cases.parquet


In [1]:
#!pip install --upgrade numpy

In [2]:
spark

In [8]:
feats = spark.read.parquet("Jingjing/pca/lpd_cases.parquet")

In [12]:
from pyspark.ml.clustering import KMeans

In [13]:
count = feats.count()
clusters = int((count) * (0.01))
print clusters

13284


In [11]:
feats.printSchema()

root
 |-- 0: double (nullable = true)
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- 4: double (nullable = true)
 |-- 5: double (nullable = true)
 |-- 6: double (nullable = true)
 |-- 7: double (nullable = true)
 |-- 8: double (nullable = true)
 |-- 9: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)
 |-- 13: double (nullable = true)
 |-- 14: double (nullable = true)
 |-- 15: double (nullable = true)
 |-- 16: double (nullable = true)
 |-- 17: double (nullable = true)
 |-- 18: double (nullable = true)
 |-- 19: double (nullable = true)
 |-- 20: double (nullable = true)
 |-- 21: double (nullable = true)
 |-- 22: double (nullable = true)
 |-- 23: double (nullable = true)
 |-- 24: double (nullable = true)
 |-- 25: double (nullable = true)
 |-- 26: double (nullable = true)
 |-- 27: double (nullable = true)
 |-- 28: double (nullable = true)
 |-- 29: double (nu

In [16]:
kmeans = KMeans().setFeaturesCol("pcaFeatures").setPredictionCol("kmeanPred").setK(clusters)
kmeansModel = kmeans.fit(feats)
labels = kmeansModel.transform(feats)

In [19]:
labels.take(5)

[Row(0=7.672204069619733, 1=23.139551155589572, 2=12.439371057348891, 3=11.767803884266275, 4=6.497767771361338, 5=6.078759033035832, 6=4.877647882355185, 7=4.9826481446123605, 8=6.607446642415919, 9=5.487435134055554, 10=4.93895194844217, 11=5.153093423092696, 12=5.171347597365264, 13=4.498202349819353, 14=4.021524387112349, 15=3.918495058747482, 16=47.31671001682913, 17=33.929748936768114, 18=79.3796738554151, 19=77.57462393306233, 20=9.205018245382506, 21=7.007729046653032, 22=8.795933361509991, 23=5.717792089617699, 24=4.634559686364599, 25=4.219548913662469, 26=4.654634001003799, 27=4.563989609727104, 28=2.8403032122874485, 29=2.66592440696545, 30=2.833837529899225, 31=3.073371521265401, 32=665712.37189324, 33=3400659.4853929714, 34=1565270.6413926089, 35=1268963.1635622182, 36=81111.144575215, 37=452493.31342572096, 38=121947.76240025435, 39=101090.15881622967, 40=24736.72096118546, 41=80151.76233653785, 42=30031.374855207534, 43=28423.143906926096, 44=6313.6680134978, 45=12769.2

In [21]:
from operator import add

kmeansRdd=(labels.select('kmeanPred', labels.explode('pcaFeatures'))
 .rdd
 .map(lambda x: (x, 1))
 .reduceByKey(add)
 .groupBy(lambda x: x[0][0])
 .mapValues(lambda x: [r[1] for r, _ in sorted(x, key=lambda a: -a[1])[:2]])
 .collect())



AttributeError: 'DataFrame' object has no attribute 'explode'

In [27]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
from pyspark.mllib.linalg import SparseVector, DenseVector

window = Window.partitionBy(labels['kmeanPred']).orderBy(labels['kmeanPred'].desc())
# kmeansGroup=labels.select('kmeanPred',rank().over(window).alias('rank')).filter(col('rank') >= 100).show()
kmeansGroup=labels.select('kmeanPred',rank().over(window).alias('rank')).show()

AnalysisException: u'Window function rank() requires window to be ordered, please add ORDER BY clause. For example SELECT rank()(value_expr) OVER (PARTITION BY window_partition ORDER BY window_ordering) from table;'

In [26]:
labels.count()
#labels.select('*').take(5)
labelGroup=labels.groupby('kmeanPred')
labelGroup.take(3)

AttributeError: 'GroupedData' object has no attribute 'take'

In [9]:
print "Cluster Centers: "
#print labels.select("kmeanPred","timeId","patId","pcaFeatures").take(2)
print labels.select("kmeanPred","timeId","pcaFeatures").take(2)

Cluster Centers: 
[Row(kmeanPred=6730, timeId=2808908611584, pcaFeatures=DenseVector([-2.8089, -4848254.1445, 596874.2149, 2378065.3254, 3902076.5353, -3944635.9067, -2693228.3459, 4848467.2371, -3203208.348, -1453027.2485, -191815.1546, -1772945.8638, 1579974.9144, -5841591.3071, 2148839.8022, 364526.6722, -874567.4394, 1073854.9402, 2623507.395, -3149571.1975, 167625.5121, 696103.2274, 99421.8737, -916098.9796, -372161.6635, -2388567.5418, -2245223.4847, 2160163.909, -1246333.765, -14963.9503, -126508.5012, -265137.639, 330263.7039, -1361190.691, 284721.6617, 121363.5056, -113854.9883, -314393.4298, -685788.2257, 246426.268, 640991.9553, 383185.1501, 383803.9201, -40589.3043, 110894.0119, -355529.7551, 41535.2014, 295262.0323, 26851.9543, -16.5407, 62.0553, -148.9363, -27.8699, 1.3879, 2.9154, 14.9392, 31.0383, -50.1794, -33.7898, -19.3298, -114.3652, 10.2939, -194.4644, 26.3798, -16.543, -0.3154, -42.9809, -0.4427, 25.6819, -12.0316, -19.0046, 12.3635, -13.0018, 4.0976, -12.6545, -3

In [11]:
labels.printSchema()

root
 |-- 0: double (nullable = true)
 |-- 1: double (nullable = true)
 |-- 2: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- 4: double (nullable = true)
 |-- 5: double (nullable = true)
 |-- 6: double (nullable = true)
 |-- 7: double (nullable = true)
 |-- 8: double (nullable = true)
 |-- 9: double (nullable = true)
 |-- 10: double (nullable = true)
 |-- 11: double (nullable = true)
 |-- 12: double (nullable = true)
 |-- 13: double (nullable = true)
 |-- 14: double (nullable = true)
 |-- 15: double (nullable = true)
 |-- 16: double (nullable = true)
 |-- 17: double (nullable = true)
 |-- 18: double (nullable = true)
 |-- 19: double (nullable = true)
 |-- 20: double (nullable = true)
 |-- 21: double (nullable = true)
 |-- 22: double (nullable = true)
 |-- 23: double (nullable = true)
 |-- 24: double (nullable = true)
 |-- 25: double (nullable = true)
 |-- 26: double (nullable = true)
 |-- 27: double (nullable = true)
 |-- 28: double (nullable = true)
 |-- 29: double (nu

In [19]:
clus=kmeansModel.clusterCenters()

In [33]:
import numpy as np
print(clus[0].shape,len(clus))
na=np.array(clus)
np.savetxt('cluster_pts.csv',na)



((100,), 15007)


In [1]:
print(na.shape)

NameError: name 'na' is not defined

In [ ]:
kmeansModel.save("Jingjing/kmeans/100cases_2.parquet")

KeyboardInterrupt: 